In [1]:
import torch
import torch.nn as nn


### Bench model

In [2]:
class Model4_1(nn.Module):
    def __init__(self):
        super(Model4_1, self).__init__()
        self.lin1 = nn.Linear(784, 100)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(100, 10)

    def forward(self, x):
        out = self.lin1(x)
        out = self.relu(out)
        out = self.lin2(out)
        return out
model4_1 = Model4_1()

In [4]:
class Model4_2(nn.Module):
    def __init__(self):
        super(Model4_2, self).__init__()
        self.lin1 = nn.Linear(784, 100)
        self.tanh = nn.Tanh()
        self.lin2 = nn.Linear(100, 10)

    def forward(self, x):
        out = self.lin1(x)
        out = self.tanh(out)
        out = self.lin2(out)
        return out
model4_2 = Model4_2()

In [5]:
class Model4_3(nn.Module):
    def __init__(self):
        super(Model4_3, self).__init__()
        self.lin1 = nn.Linear(784, 100)
        self.sigmoid = nn.Sigmoid()
        self.lin2 = nn.Linear(100, 10)

    def forward(self, x):
        out = self.lin1(x)
        out = self.sigmoid(out)
        out = self.lin2(out)
        return out
model4_3 = Model4_3()

In [6]:
import torchvision
import torchvision.transforms as transforms
batchSize = 100
trainSet = torchvision.datasets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
trainLoader = torch.utils.data.DataLoader(dataset=trainSet, batch_size=batchSize, shuffle = True)
testSet = torchvision.datasets.MNIST(root='./data', train = False, transform=transforms.ToTensor(), download=True)
testLoader = torch.utils.data.DataLoader(dataset=testSet, batch_size=batchSize, shuffle = True)

In [9]:
def accuracy(testLoader,model):
    correct, total = 0, 0
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            outputs = model(images.view(-1, 28*28))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return(correct / total)

In [10]:
import torch.optim as optim
import time

def benchmark(trainLoader, model, epochs=1, lr=0.01):
    model.__init__()
    start = time.time()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(trainLoader):
            optimizer.zero_grad()
            outputs = model(images.view(-1, 28 * 28))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    print("Accuracy: {0:.4f}".format(accuracy(testLoader, model)))
    print("Training Time: {0:.2f}".format(time.time() - start))

In [11]:
print('ReLU activation:')
benchmark(trainLoader, model4_1, epochs=5, lr = 0.1)
print('Tanh activation')
benchmark(trainLoader, model4_2, epochs=5, lr = 0.1)
print('sigmoid activation')
benchmark(trainLoader, model4_3, epochs=5, lr = 0.1)

ReLU activation:
Accuracy: 0.9577
Training Time: 27.80
Tanh activation
Accuracy: 0.9516
Training Time: 32.06
sigmoid activation
Accuracy: 0.9201
Training Time: 34.36


In [14]:
class Model4_4(nn.Module):
    def __init__(self):
        super(Model4_4, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(784, 100), nn.ReLU())
        self.layer2 = nn.Sequential(nn.Linear(100, 50), nn.ReLU(), nn.Linear(50, 10))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out
model4_4 = Model4_4()

In [15]:
print('Mutli Linear Layers:')
benchmark(trainLoader, model4_4, epochs=5, lr = 0.1)

Mutli Linear Layers:
Accuracy: 0.9663
Training Time: 30.42


In [17]:
class CNNModel1(nn.Module):
    def __init__(self):
        super(CNNModel1, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.maxPool1 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxPool1(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [19]:
def benchmark(trainLoader, model, epochs=1, lr=0.01):
    model.__init__()
    start=time.time()
    optimiser = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(trainLoader):
            optimiser.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()
    print('Accuracy: {0:.4f}'.format(accuracy(testLoader,model)))
    print('Training time: {0:.2f}'.format(time.time() - start))

def accuracy(testLoader,model):
    correct, total = 0, 0
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return(correct / total)

In [20]:
print('1 convolutional layers')
model1 = CNNModel1()
benchmark(trainLoader, model1, lr=0.1)

1 convolutional layers
Accuracy: 0.9691
Training time: 37.15


In [25]:
class CNNModel2(nn.Module):
    def __init__(self):
        super(CNNModel2, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [26]:
print('2 convolutional layers')
model2 = CNNModel2()
benchmark(trainLoader, model2, lr=0.1)

2 convolutional layers
Accuracy: 0.9721
Training time: 30.94
